In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import math

In [65]:
draft = {}
for i in range (6,22):
    if (i<10):
        year = ("200"+str(i))
    else:
        year = ("20"+str(i))

    draft[i]=(pd.read_csv(year+'_draft.csv', header=1))
    draft[i]['Year'] = int(year)

In [66]:
draft_df = pd.concat(draft)
draft_df.to_csv('D:/3A\MSCI 446 Intro to ML\Proj\data.txt')

# Getting Data


In [57]:
agility_stats = pd.read_csv('agility_stats.csv')
anthro_stats = pd.read_csv('anthro_stats.csv')
eff_stats = pd.read_csv('eff_stats.csv')
game_stats = pd.read_csv('game_stats.csv')

In [58]:
data = [agility_stats, anthro_stats, eff_stats, game_stats]
names = ['agility_stats', 'anthro_stats', 'eff_stats', 'game_stats']

## Initial Exploration


In [ ]:
for i in range(0,4):
  print (names[i])
  print (data[i].head())
  print ('='*100)
  print ('')

In [59]:
for i in range(0,4):
  print (names[i])
  print (data[i].columns)
  print ('='*100)
  print ('')

agility_stats
Index(['PLAYER', 'POS', 'LANE_AGILITY_TIME', 'SHUTTLE_RUN',
       'THREE_QUARTER_SPRINT', 'STANDING_VERTICAL_LEAP', 'MAX_VERTICAL_LEAP',
       'MAX_BENCH_PRESS', 'YEAR'],
      dtype='object')

anthro_stats
Index(['PLAYER', 'POS', 'BODY_FAT', 'HAND_LENGTH_INCHES', 'HAND_WIDTH_INCHES',
       'HEIGHT_WO_SHOES', 'HEIGHT_W_SHOES', 'STANDING_REACH', 'WEIGHT',
       'WINGSPAN', 'YEAR'],
      dtype='object')

eff_stats
Index(['#', 'PLAYER', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'EFF', 'YEAR'],
      dtype='object')

game_stats
Index(['Column1', '_1', 'Rk', 'Pk', 'Tm', 'PLAYER', 'College', 'Yrs', 'G',
       'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1',
       'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Year'],
      dtype='object')



## Removing unecessary columns

As seen from the vast number of columns above, lets remove uncessary columns that provide no value for example:

- Column1 - all the number 6
- _1 - index of players that does not provide value
- Tm - College team
- MP - Total minutes played (we will use MP/ game)

In [43]:
game_stats = game_stats.drop(columns=['Column1', '_1', 'Rk', 'Pk', 'Tm', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST'], axis = 1)
eff_stats = eff_stats[['#','PLAYER','EFF']]
agility_stats = agility_stats.drop(columns=['POS', 'YEAR'], axis = 1)
anthro_stats = anthro_stats.drop(columns=['POS','YEAR'], axis = 1)

## Removing duplicate players


In [44]:
#reset data list with changes
data = [agility_stats, anthro_stats, eff_stats, game_stats]

Let's take a look at the current available columns and how much data they hold

In [60]:
for i in range(0,4):
  print (names[i])
  print (data[i].info())
  print ('='*100)
  print ('')

agility_stats
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   PLAYER                  1004 non-null   object
 1   POS                     1000 non-null   object
 2   LANE_AGILITY_TIME       1004 non-null   object
 3   SHUTTLE_RUN             1004 non-null   object
 4   THREE_QUARTER_SPRINT    1004 non-null   object
 5   STANDING_VERTICAL_LEAP  1004 non-null   object
 6   MAX_VERTICAL_LEAP       1004 non-null   object
 7   MAX_BENCH_PRESS         1004 non-null   object
 8   YEAR                    1004 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 70.7+ KB
None

anthro_stats
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PLAYER              1004 non-n

In [66]:
#merge all datasets on name columns
#left joins are made as players without efficiency cannot be used as training data
df_all = eff_stats.merge(game_stats, how='left', on='PLAYER').merge(agility_stats, how='left', on='PLAYER').merge(anthro_stats, how='left', on='PLAYER').merge(agility_stats, how='left', on='PLAYER')

In [67]:
df_all.shape

(895, 35)

In [71]:
df_all['PLAYER'].unique().size

754

Looks like we have a few duplicate players, lets save the first appearance of each player so that the data is not over represented and the model is not skewed.

In [76]:
df_all = df_all.drop_duplicates('PLAYER')
df_all = df_all.drop(columns=['#'])

In [77]:
df_all.describe()

,EFF,FG%,3P%,FT%,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Year
count,754.000000,524.000000,508.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.00000,524.000000,524.000000
mean,7.199867,0.448756,0.304380,0.736817,20.443130,8.591412,3.619656,1.845420,15.496565,0.075303,-1.51355,3.834733,2013.578244
std,4.048345,0.062112,0.092633,0.092023,7.599485,5.010633,2.012432,1.564496,22.132155,0.053959,2.43050,9.108745,4.840040
min,0.200000,0.267000,0.000000,0.390000,3.200000,0.900000,0.300000,0.100000,-1.800000,-0.203000,-10.30000,-3.900000,2006.000000
25%,4.225000,0.411000,0.286000,0.682000,14.600000,4.800000,2.200000,0.800000,1.200000,0.041750,-3.10000,-0.400000,2009.000000
50%,6.300000,0.440000,0.330000,0.750000,20.300000,7.400000,3.150000,1.300000,6.350000,0.075500,-1.60000,0.300000,2013.000000
75%,9.400000,0.475250,0.360000,0.805000,26.725000,11.325000,4.500000,2.400000,21.925000,0.108250,0.00000,4.400000,2018.000000
max,25.600000,0.720000,0.667000,0.915000,36.700000,27.100000,13.300000,9.100000,152.200000,0.227000,6.80000,74.800000,2021.000000


## Interesting things to note:

The following are medians the median statistics. Keep in mind these are higher than the average college player as all of thesee players were selected to play in the NBA

EFF	FG%	3P%	FT%	MP.1	PTS.1	TRB.1	AST.1	WS	WS/48	BPM	VORP	Year
6.300000	0.440000	0.330000	0.750000	20.300000	7.400000	3.150000	1.300000	6.350000	0.075500	-1.60000	0.300000

In [78]:
df_all.to_csv('D:/3A\MSCI 446 Intro to ML\Proj\data_all.txt')

In [2]:
#For future use so as not to run all cells
df = pd.read_csv('data_all.csv')

## Data Types

Lets make sure all data types are as they should be: numeric vs categorical

In [ ]:
df.info()

Looks like all data that is not from game_stats are of type object. 
All of these need to be changed to numerical

 0   Column1                   754 non-null    int64  
 1   PLAYER                    754 non-null    object 
 2   EFF                       754 non-null    float64
 3   FG%                       524 non-null    float64
 4   3P%                       508 non-null    float64
 5   FT%                       524 non-null    float64
 6   MP.1                      524 non-null    float64
 7   PTS.1                     524 non-null    float64
 8   TRB.1                     524 non-null    float64
 9   AST.1                     524 non-null    float64
 10  WS                        524 non-null    float64
 11  WS/48                     524 non-null    float64
 12  BPM                       524 non-null    float64
 13  VORP                      524 non-null    float64
 14  Year                      524 non-null    float64
 15  LANE_AGILITY_TIME_x       450 non-null    object 
 16  SHUTTLE_RUN_x             450 non-null    object 
 17  THREE_QUARTER_SPRINT_x    450 non-null    object 
 18  STANDING_VERTICAL_LEAP_x  450 non-null    object 
 19  MAX_VERTICAL_LEAP_x       450 non-null    object 
 20  MAX_BENCH_PRESS_x         450 non-null    object 
 21  BODY_FAT                  450 non-null    object 
 22  HAND_LENGTH_INCHES        450 non-null    object 
 23  HAND_WIDTH_INCHES         450 non-null    object 
 24  HEIGHT_WO_SHOES           437 non-null    object 
 25  HEIGHT_W_SHOES            437 non-null    object 
 26  STANDING_REACH            437 non-null    object 
 27  WEIGHT                    450 non-null    object 
 28  WINGSPAN                  438 non-null    object 
 29  LANE_AGILITY_TIME_y       450 non-null    object 
 30  SHUTTLE_RUN_y             450 non-null    object 
 31  THREE_QUARTER_SPRINT_y    450 non-null    object 
 32  STANDING_VERTICAL_LEAP_y  450 non-null    object 
 33  MAX_VERTICAL_LEAP_y       450 non-null    object 
 34  MAX_BENCH_PRESS_y         450 non-null    object 

## Manual Data Cleaning
Manually retrieved and filled in missing data for player who are missing college stats but not anthro stats as anthro is more difficult to find
Many players played in the NBA and NCAA but were not drafted immediately and as a result did not appear in our original data search. These players were manually tracked down and updated.

## Regex

Looking at length measurements, all the attributes are in feet and inches for example: 7'2.75'' or 6'10.25''   
Unfortunately, the decimals are inconsistent so we will need to parse out the values and convert them into cm following the format of   
Value ' value ''    
where the first value is multiplied by 30 and the second value is multiplied by 2.54 (cm/inch)

In [26]:
measurement = "7'2.75''"
splt = measurement.split("'")
cm = float(splt[0])*30+float(splt[1])*2.54
type(cm)

float

In [54]:
def to_cm(measurement):
    #Check to see if it is NaN
    if measurement == measurement: 
        splt = str(measurement).split("'")
        cm = float(splt[0])*30+float(splt[1])*2.54
        return cm
    else:
        pass

In [47]:
df2 = df[['HEIGHT_WO_SHOES','PLAYER']]
print(1,type(df2['HEIGHT_WO_SHOES'][0]))
print(type(df2['HEIGHT_WO_SHOES'][3]))

1 <class 'float'>
<class 'str'>


In [56]:
df2['test'] = df2['HEIGHT_WO_SHOES'].apply(to_cm)
df2.head()

<ipython-input-56-e313f74c4c76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['test'] = df2['HEIGHT_WO_SHOES'].apply(to_cm)


,HEIGHT_WO_SHOES,PLAYER,test
0,NaN,Cade Cunningham,NaN
1,NaN,Franz Wagner,NaN
2,NaN,Jalen Green,NaN
3,6'7.0'',Scottie Barnes,197.78
4,NaN,Evan Mobley,NaN


In [7]:
#All these columns are of type object
columns = ['LANE_AGILITY_TIME_x','SHUTTLE_RUN_x','THREE_QUARTER_SPRINT_x', 'STANDING_VERTICAL_LEAP_x',
	'MAX_VERTICAL_LEAP_x','MAX_BENCH_PRESS_x','BODY_FAT','HAND_LENGTH_INCHES','HAND_WIDTH_INCHES',
    'HEIGHT_WO_SHOES','HEIGHT_W_SHOES','STANDING_REACH','WEIGHT','WINGSPAN','LANE_AGILITY_TIME_y',
    'SHUTTLE_RUN_y','THREE_QUARTER_SPRINT_y','STANDING_VERTICAL_LEAP_y','MAX_VERTICAL_LEAP_y','MAX_BENCH_PRESS_y']


In [20]:
df["BODY_FAT"] = [str(x).replace('-%', '') for x in df['BODY_FAT']]
df["BODY_FAT"]= pd.to_numeric(df["BODY_FAT"],errors='coerce')
# type(df["BODY_FAT"][2])
for column in columns:
    df[column] = [str(x).replace('-', '') for x in df[column]]
    df[column]= pd.to_numeric(df[column],errors='coerce')

df.head()
#Nice it works

,Column1,PLAYER,EFF,FG%,3P%,FT%,MP.1,PTS.1,TRB.1,AST.1,...,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN,LANE_AGILITY_TIME_y,SHUTTLE_RUN_y,THREE_QUARTER_SPRINT_y,STANDING_VERTICAL_LEAP_y,MAX_VERTICAL_LEAP_y,MAX_BENCH_PRESS_y
0,0,Cade Cunningham,15.9,0.400,0.320,0.839,31.9,16.2,5.7,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Franz Wagner,15.8,0.468,0.366,0.852,31.9,15.7,4.6,2.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Jalen Green,11.1,0.396,0.312,0.811,31.0,15.0,3.1,2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Scottie Barnes,18.5,0.490,0.314,0.741,35.3,14.9,7.6,3.2,...,NaN,NaN,225.4,NaN,10.88,2.99,3.15,36.0,39.5,NaN
4,4,Evan Mobley,18.5,0.499,0.257,0.680,34.1,14.7,8.1,2.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Number of rows that are missing a value
df.shape[0] - df.dropna().shape[0]

754

Bad news, every single row is missing at least one value.

# Linear Regression

In [4]:
# Import for Linear Regression
import sklearn
from sklearn.linear_model import LinearRegression
df = pd.read_csv('data_all.csv')

In [5]:
# Create a LinearRegression Object
lreg = LinearRegression()

In [8]:
# Data Columns
X_multi = df.drop(['PLAYER','EFF','Year'],1)

# Targets
Y_target = df['EFF']

In [ ]:
# Implement Linear Regression
lreg.fit(X_multi,Y_target)

In [ ]:
# Set a DataFrame from the Features
coeff_df = pd.DataFrame(df.columns)
coeff_df.columns = ['Features']

# Set a new column lining up the coefficients from the linear regression
coeff_df['Coefficient Estimate'] = pd.Series(lreg.coef_)

# Show
coeff_df

In [ ]:
#Cross validation to reduce overfitting
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lreg,X_multi,df['EFF'],cv=10,scoring='explained_variance')
scores

In [ ]:
#Training and validation

# Grab the output and set as X and Y test and train data sets!
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_multi,Y_target,test_size = 0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

# Create our regression object
lreg = LinearRegression()

# Once again do a linear regression, except only on the training sets this time
lreg.fit(X_train,Y_train)

In [ ]:
# Predictions on training and testing sets
pred_train = lreg.predict(X_train)
pred_test = lreg.predict(X_test)

In [ ]:
print("Fit a model X_train, and calculate MSE with Y_train: %.2f"  %np.mean((Y_train - pred_train) **2) )
    
print("Fit a model X_train, and calculate MSE with X_test and Y_test: %.2f"  %np.mean((Y_test - pred_test) **2) )